# Ejercicio 2 Regresion
En este primer bloque solo cargamos paqueterias necesarias y leeremos el archivo con la base de datos ya limpiada que se realizo en el ejercicio 1, solo se dejo con la base de datos  con las variables que tuvieron una menor correlacion 

In [73]:
# Cargar los paquetes
using DataFrames
using CSV
using StatsBase
using GLM
data = DataFrame(CSV.File("bottle_EDA.csv"))

637151×9 DataFrame
    Row │ Depthm   Salnty   STheta   R_Depth  R_SALINITY  R_SIGMA  R_DYNHT  R_ ⋯
        │ Float64  Float64  Float64  Float64  Float64     Float64  Float64  Fl ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │     8.0  33.44    25.656       8.0      33.44    25.65     0.01      ⋯
      2 │    10.0  33.437   25.654      10.0      33.437   25.65     0.02
      3 │    19.0  33.42    25.643      19.0      33.42    25.64     0.04
      4 │    20.0  33.421   25.643      20.0      33.421   25.64     0.04
      5 │    30.0  33.431   25.651      30.0      33.431   25.65     0.07      ⋯
      6 │    39.0  33.44    25.658      39.0      33.44    25.65     0.09
      7 │    50.0  33.424   25.682      50.0      33.424   25.68     0.11
      8 │    58.0  33.42    25.71       58.0      33.42    25.71     0.13
   ⋮    │    ⋮        ⋮        ⋮        ⋮         ⋮          ⋮        ⋮        ⋱
 637145 │    33.0  33.3172  24.4465     33.0      33.317   24.447    0.133     ⋯
 637146 │    10.0  33.548   23.6093     10.0      33.548   23.609    0.043
 637147 │   170.0  33.804   26.153     170.0      33.804   26.153    0.532
 637148 │   201.0  33.9538  26.3388    201.0      33.954   26.339    0.588
 637149 │    12.0  33.3802  24.0439     12.0      33.38    24.044    0.047     ⋯
 637150 │    40.0  33.3042  24.3351     40.0      33.304   24.335    0.154
 637151 │   101.0  33.211   24.9583    101.0      33.211   24.958    0.379
                                               2 columns and 637136 rows omitted

Aplicamos una regresion lineal con todo los datos

In [74]:
model = lm(@formula(T_degC ~ Depthm + Salnty + STheta + R_Depth + R_SALINITY + R_SIGMA + R_DYNHT + R_PRES), data)
predicted = predict(model)

# Residuos (observado - predicho)
residuals = data.T_degC .- predicted

# Suma de errores (suma de los cuadrados de los residuos)
suma_errores = sum(residuals .^ 2)

println("La suma de errores es: $suma_errores")
println(model)

La suma de errores es: 35193.58757927555
StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

T_degC ~ 1 + Depthm + Salnty + STheta + R_Depth + R_SALINITY + R_SIGMA + R_DYNHT + R_PRES

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error        t  Pr(>|t|)   Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)  10.6201      0.0327318    324.46    <1e-99  10.5559     10.6842
Depthm        0.00209433  0.0113027      0.19    0.8530  -0.0200585   0.0242472
Salnty       -0.0814453   0.760691      -0.11    0.9147  -1.57238     1.40948
STheta       -4.50936     0.0999481    -45.12    <1e-99  -4.70526    -4.31347
R_Depth       0.169597    0.0113562     14.93    <1e-49   0.147339    0.191855
R_SALINITY    3.71631     0.7

Podemos ver que algunas variables como Depthm, Salnty, R_Sigma tienen p-values demasiado altos, comunmente se toma ltos cuando son mayores a 0.05, es decir tenemos evidencia suficiente para poder descartar estas variables, ya que no aportan tanto al modelo como las otras, es decir no son significativas y solo ocasionara un overfititting.

Volvemos a hacer la regresion pero ahora sin las variables que mencionamos anteriormente

In [75]:
model = lm(@formula(T_degC ~  STheta + R_Depth + R_SALINITY  + R_DYNHT + R_PRES), data)
predicted = predict(model)

# Residuos (observado - predicho)
residuals = data.T_degC .- predicted

# Suma de errores (suma de los cuadrados de los residuos)
suma_errores = sum(residuals .^ 2)

println("La suma de errores es: $suma_errores")
println(model)

La suma de errores es: 35193.87939373516
StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

T_degC ~ 1 + STheta + R_Depth + R_SALINITY + R_DYNHT + R_PRES

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                 Coef.   Std. Error         t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  10.6194    0.0327266      324.49    <1e-99  10.5552    10.6835
STheta       -4.73657   0.000637015  -7435.57    <1e-99  -4.73782   -4.73532
R_Depth       0.171724  0.00106672     160.98    <1e-99   0.169633   0.173815
R_SALINITY    3.63491   0.00101939    3565.75    <1e-99   3.63291    3.63691
R_DYNHT       0.807869  0.00461719     174.97    <1e-99   0.79882    0.816919
R_PRES       -0.174847  0.00105635    -165.52    <1e-99  -0.176917  -

Vemos que el la suma de errores es muy simmilar al modelo anteiror pero simmplificamos el modelo.

Vemos que variables se autodescriben o son redundantes al tener una correlacion alta, para poder simplificar el modelo y tener menos variables, se dejan afuera  las que su correlacion sea mayor a 0.99

In [76]:
for i in ["STheta", "R_Depth", "R_SALINITY", "R_DYNHT", "R_PRES"]
    # Especificar la columna objetivo y el umbral de correlación
    try
        col_target = i
        threshold1 = 0.99
    
        # Calcular las correlaciones
        correlations = map(col -> cor(data[!, col], data[!, col_target]), names(data))
    
        # Filtrar las columnas con alta correlación
        high_corr_columns = names(data)[[ c< threshold1 && col != col_target for (c, col) in zip(correlations, names(data))]]
        push!(high_corr_columns,i)
        # Crear un nuevo DataFrame con las columnas seleccionadas
        data = data[:, high_corr_columns]
    catch
    end
end
data

637151×5 DataFrame
    Row │ T_degC   STheta   R_Depth  R_SALINITY  R_DYNHT 
        │ Float64  Float64  Float64  Float64     Float64 
────────┼────────────────────────────────────────────────
      1 │  10.46   25.656       8.0      33.44     0.01
      2 │  10.46   25.654      10.0      33.437    0.02
      3 │  10.45   25.643      19.0      33.42     0.04
      4 │  10.45   25.643      20.0      33.421    0.04
      5 │  10.45   25.651      30.0      33.431    0.07
      6 │  10.45   25.658      39.0      33.44     0.09
      7 │  10.24   25.682      50.0      33.424    0.11
      8 │  10.06   25.71       58.0      33.42     0.13
   ⋮    │    ⋮        ⋮        ⋮         ⋮          ⋮
 637145 │  16.045  24.4465     33.0      33.317    0.133
 637146 │  20.181  23.6093     10.0      33.548    0.043
 637147 │   9.219  26.153     170.0      33.804    0.532
 637148 │   8.791  26.3388    201.0      33.954    0.588
 637149 │  17.959  24.0439     12.0      33.38     0.047
 637150 │  16.492  24.3351     40.0      33.304    0.154
 637151 │  13.277  24.9583    101.0      33.211    0.379
                                      637136 rows omitted

Volvemos a aplicar el modelo pero con las neuvas variables que no estan tan correlacionadas y  logramos simplificar aun mas el modelo con una variable menos

In [77]:
model = lm(@formula(T_degC ~ STheta +R_Depth + R_SALINITY+ R_DYNHT), data)
predicted = predict(model)

# Residuos (observado - predicho)
residuals = data.T_degC .- predicted

# Suma de errores (suma de los cuadrados de los residuos)
suma_errores = sum(residuals .^ 2)

println("La suma de errores es: $suma_errores")
println(model)

La suma de errores es: 36707.20235441297
StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

T_degC ~ 1 + STheta + R_Depth + R_SALINITY + R_DYNHT

Coefficients:
───────────────────────────────────────────────────────────────────────────────────
                   Coef.   Std. Error         t  Pr(>|t|)    Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────
(Intercept)  10.6983      0.0334192      320.12    <1e-99  10.6328      10.7638
STheta       -4.74488     0.000648543  -7316.21    <1e-99  -4.74615     -4.74361
R_Depth      -0.00483414  8.32054e-6    -580.99    <1e-99  -0.00485044  -0.00481783
R_SALINITY    3.63928     0.00104073    3496.86    <1e-99   3.63724      3.64132
R_DYNHT       1.11011     0.004331       256.32    <1e-99   1.10162      1.11859
───────────────────────────────────────

EL modelo sigue teniendo un error similar a los anteriores